In [1]:
import warnings
warnings.filterwarnings('ignore')

from TGA.utils import Dataset, GraphsizePretrained
from tqdm import tqdm
from tqdm.notebook import tqdm
from time import time
import numpy as np

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/mangaravite/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
pretrained_dir='/home/mangaravite/Documentos/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt'

In [7]:
%%time
graph_builder = GraphsizePretrained(w=1, verbose=True,
                   pretrained_vec=pretrained_dir)

400000it [00:29, 13724.66it/s]


CPU times: user 24 s, sys: 867 ms, total: 24.8 s
Wall time: 29.3 s


In [10]:
from etc.base import Dataset

In [12]:
dataset = Dataset('20ng')
fold = dataset.get_fold(0, 10, with_val=False)
fold._fields, len(fold.X_train)

http://homepages.dcc.ufmg.br/~vitormangaravite/.etc/datasets/split_10.csv
bla


(('X_train', 'y_train', 'X_test', 'y_test'), 16961)

In [13]:
import torch
import dgl
import dgl.function as fn
import torch.nn as nn
import torch.nn.functional as F

import networkx as nx
from dgl.nn.pytorch.conv import GraphConv, GATConv
from dgl.nn.pytorch.glob import GlobalAttentionPooling

from sklearn.preprocessing import LabelEncoder

from itertools import repeat

import torch.optim as optim
from torch.utils.data import DataLoader

Using backend: pytorch


OSError: libcublas.so.10: cannot open shared object file: No such file or directory

In [9]:
%%time
graph_builder.fit(fold.X_train, fold.y_train)

100%|██████████| 16954/16954 [00:14<00:00, 1139.51it/s]


CPU times: user 18.8 s, sys: 154 ms, total: 18.9 s
Wall time: 18.9 s


GraphsizePretrained(encoding=None,
                    pretrained_vec='/home/Universidade/LBD/pretrained_vectors/glove/glove.6B.300d.txt',
                    verbose=None, w=1)

In [10]:
len(graph_builder.g.edges), len(graph_builder.g)

(273731, 56503)

In [11]:
list(map(lambda x: (x,graph_builder.g.degree()[x]), graph_builder.label_ids))

[(0, 11512),
 (1, 7455),
 (2, 8667),
 (3, 8889),
 (4, 9347),
 (5, 13593),
 (6, 13216),
 (7, 10346),
 (8, 7524),
 (9, 9752),
 (10, 10433),
 (11, 14732),
 (12, 12305),
 (13, 10920),
 (14, 9115),
 (15, 9008),
 (16, 9719),
 (17, 12025),
 (18, 13315),
 (19, 15395)]

In [12]:
class GenericGAT(nn.Module):
    def __init__(self, in_dim, hidden_dim,
                 drop=.5, n_heads=8, attn_drop=.5,
                 activation=F.leaky_relu, n_convs=2,
                 first_hidden='emb', encoders={'term','label'},
                 device='cpu:0'):
        super(GenericGAT, self).__init__()
        self.device = torch.device(device)
        self.first_hidden = first_hidden
        
        self.encoders = nn.ModuleDict({
            k: nn.Linear(in_dim, hidden_dim).to(self.device) for k in encoders
        })
        
        self.layers = nn.ModuleList([
            GATConv(hidden_dim, hidden_dim, residual=True, num_heads=n_heads, activation=activation,
                    feat_drop=drop, attn_drop=attn_drop).to(self.device) for _ in range(n_convs)
        ])
        self.down_proj = [
            nn.Linear(n_heads*hidden_dim, hidden_dim).to(self.device) for _ in range(n_convs)
        ]
        
    def forward(self, G, **kwargs):
        with G.local_scope():
            h = G.ndata[self.first_hidden].float()
            for (k, mask) in kwargs.items():
                if k in self.encoders:
                    if mask is not None:
                        h[ mask ] = self.encoders[k]( h[ mask ] )
                    else:
                        h = self.encoders[k]( h )

            for l, conv in enumerate(self.layers):
                h = conv(G, h)
                h = h.view(h.shape[0], -1)
                h = self.down_proj[l]( h )
                #h = F.relu( h )
        return h
        

In [9]:
def collate(param):
    X, y = zip(*param)
    Gs_nx = graph_builder.transform(X)
    
    Gs_dgl_list = []
    for g in Gs_nx:
        g_dgl = dgl.DGLGraph()
        if len(g) > 0:
            g_dgl.from_networkx(g, node_attrs=['emb', 'idx'] )
        Gs_dgl_list.append( g_dgl )
    
    Gs_dgl = dgl.batch(Gs_dgl_list)
    
    big_graph_dgl = dgl.DGLGraph()
    big_graph_dgl.from_networkx(graph_builder.g, node_attrs=['emb', 'label', 'idx'] )
    
    #subgraph = graph_builder.g.subgraph(idx_terms)
    #big_graph_dgl.from_networkx(subgraph, node_attrs=['emb', 'label', 'idx'] )
    
    return big_graph_dgl, Gs_dgl, torch.tensor(y)

In [10]:
class CosineDistanceLoss(torch.nn.Module):
    def __init__(self, reduction='mean', alpha=-1.):
        super(CosineDistanceLoss, self).__init__()

        self.loss = nn.CosineEmbeddingLoss(reduction=reduction)
        self.alpha = alpha

    def forward(self, X, Y=None, y_idxs=None):
        nsmpl, ndims = X.shape
        A = []
        B = []
        target = []
        if Y is not None and y_idxs is not None:
            for (x,y_idx) in zip(X, y_idxs):
                for i,y in enumerate(Y):
                    A.append( x )
                    B.append( y )
                    target.append( 1 if i == y_idx.item() else -1 )
            
        else:
            for i in range(nsmpl):
                for j in range(i+1, nsmpl):
                    A.append( X[i] )
                    B.append( self.alpha*X[j] )
                    target.append( 1 )

        A=torch.cat(A).reshape( len(target), ndims ).to(X.device)
        B=torch.cat(B).reshape( len(target), ndims ).to(X.device)
        target=torch.Tensor(target).to(X.device)
            
        
        return self.loss(A, B, target=target)

        

In [17]:
class TGA(torch.nn.Module):
    def __init__(self, input_l, hidden_l, nclass, n_heads=1,
                drop=0.5, attn_drop=0.5, loss=None,
                 device='cuda:0'):
        
        super(TGA, self).__init__()
        
        self.gat = GenericGAT( input_l, hidden_l, n_heads=n_heads,
                 drop=drop, attn_drop=attn_drop,
                 activation=None, device='cuda:0' ).to(device)
        
        self.norm_label = nn.BatchNorm1d(hidden_l).to(device)
        self.norm_terms = nn.BatchNorm1d(hidden_l).to(device)

        self.gate = nn.Linear( hidden_l, 1 ).to(device)
        self.feat = nn.Linear( hidden_l, hidden_l ).to(device)
        self.gap  = GlobalAttentionPooling(self.gate, feat_nn=self.feat).to(device)
        
        self.nclass  = nclass
        self.fc1     = nn.Linear( hidden_l, hidden_l//2 ).to(device)
        self.fc2     = nn.Linear(  hidden_l//2, self.nclass ).to(device)
        self.softmax = nn.Softmax(dim=1)
        
        self.loss = loss

    def forward(self, G, gs, y, label_idx=None):
        if label_idx is None:
            label_idx = G.ndata['label'].nonzero().flatten()
            
        terms_idx = range(len(label_idx),len(graph_builder.g))
        h_global  = self.gat(G, label=label_idx, term=terms_idx)

        h_labels            = self.norm_label(h_global[label_idx])
        h_global[terms_idx] = self.norm_terms(h_global[terms_idx])

        h_docs = self.gap( gs, h_global[gs.ndata['idx'].reshape(-1)] )
        
        h_docs_pred = h_docs
        h_docs_pred = self.fc1(h_docs_pred)
        h_docs_pred = self.fc2(h_docs_pred)
        h_docs_pred = nn.softmax(h_docs_pred, 1)
        
        return h_docs, h_docs_pred, h_labels

In [1]:
hidden_l = 64
input_l = 300
n_heads = 4
margin=-.1
drop=0.3
batch_size=64
attn_drop=0.3
device=torch.device('cuda:0')

NameError: name 'torch' is not defined

In [19]:
class TGALoss(torch.nn.Module):
    def __init__(self, sim_func=nn.CosineSimilarity(), normalized=True, margin=-1.):
        super(TGALoss, self).__init__()

        self.sim_func = sim_func
        self.margin   = margin
        self.softmax  = nn.Softmax()
        self.sigma  = nn.Parameter(torch.ones(4))
        self.crossE = nn.CrossEntropyLoss()
        self.norm = normalized

    def forward(self, X, y_pred, Y, y_idxs, explain=False):
        nsmpl, ndims, nclass = self._get_dims_(X,Y)
        
        loss_global      = self._get_global_(Y)
        norm_loss_global = loss_global/(self.sigma[0]**2) + torch.log(self.sigma[0])
        
        loss_local_pos      = self._get_local_positive_(X, Y, y_idxs)
        norm_loss_local_pos = loss_local_pos/(self.sigma[1]**2) + torch.log(self.sigma[1])
        
        loss_local_neg      = self._get_local_negative_(X, Y, y_idxs)
        norm_loss_local_neg = loss_local_neg/(self.sigma[2]**2) + torch.log(self.sigma[2])
        
        loss_crossE         = self.crossE(y_pred, y_idxs)
        norm_loss_crossE    = loss_crossE/(2*(self.sigma[3]**2)) + torch.log(self.sigma[3])
        
        if self.norm:
            loss = norm_loss_crossE + \
                    norm_loss_global.mean() + \
                    norm_loss_local_pos.mean() + \
                    norm_loss_local_neg.mean()
        else:
            loss = loss_crossE + \
                    loss_global.mean() + \
                    loss_local_pos.mean() + \
                    loss_local_neg.mean()
            
        if explain:
            explanation = {
                'global':        (loss_global.mean().item()+self.margin,    self.sigma[0].item()),
                'local_pos':     (loss_local_pos.mean().item(),             self.sigma[1].item()),
                'local_neg':     (loss_local_neg.mean().item()+self.margin, self.sigma[2].item()),
                'cross_entropy': (loss_crossE.mean().item(),                self.sigma[3].item())
            }
            return loss, explanation
        return loss
    
    def predict(self, X, Y):
        nsmpl, ndims, nclass = self._get_dims_(X,Y)
        A = []
        B = []
        for x in X:
            for y in Y:
                A.append( x )
                B.append( y )
        A=torch.cat(A).reshape( nsmpl*nclass, ndims ).to(Y.device)
        B=torch.cat(B).reshape( nsmpl*nclass, ndims ).to(Y.device)
        
        similarities = self.sim_func(A, B)
        similarities = similarities.reshape( nsmpl, nclass )
        similarities = nn.softmax(similarities, 1)
        
        return similarities
    
    def _get_global_(self, Y):
        nsmpl, ndims, nclass = self._get_dims_(None,Y)
        A = []
        B = []
        for i, y_i in enumerate(Y):
            for y_j in Y[:i]:
                A.append( y_i )
                B.append( y_j )
        A=torch.cat(A).reshape( len(A), ndims ).to(Y.device)
        B=torch.cat(B).reshape( len(B), ndims ).to(Y.device)
        
        similarities = self.sim_func(A, B)-self.margin
        similarities = torch.clamp(similarities, min=0.)
        
        return similarities
    
    def _get_local_positive_(self, X, Y, y_idxs):
        nsmpl, ndims, nclass = self._get_dims_(X,Y)
        A = []
        B = []
        for i, x in enumerate(X):
            for j, y in enumerate(Y):
                if y_idxs[i] == j:
                    A.append( x )
                    B.append( y )
                
        A=torch.cat(A).reshape( len(A), ndims ).to(Y.device)
        B=torch.cat(B).reshape( len(B), ndims ).to(Y.device)
        
        distancies = 1.-self.sim_func(A, B)
        
        return distancies
    def _get_local_negative_(self, X, Y, y_idxs):
        nsmpl, ndims, nclass = self._get_dims_(X,Y)
        A = []
        B = []
        for i, x in enumerate(X):
            for j, y in enumerate(Y):
                if y_idxs[i] != j:
                    A.append( x )
                    B.append( y )
                
        A=torch.cat(A).reshape( len(A), ndims ).to(Y.device)
        B=torch.cat(B).reshape( len(B), ndims ).to(Y.device)
        
        similarity = self.sim_func(A, B)-self.margin
        similarity = torch.clamp(similarity, min=0.)
        
        return similarity
    
    def _get_dims_(self, X, Y):
        nsmpl, ndims, nclass = None, None, None
        if X is not None:
            nsmpl, ndims  = X.shape
        if Y is not None:
            nclass, ndims = Y.shape
        return nsmpl, ndims, nclass

In [20]:
tga = TGA(input_l, hidden_l, nclass=graph_builder.n_class, loss=TGALoss(margin=margin, normalized=True),
          n_heads=n_heads, drop=drop, attn_drop=attn_drop).to(device)
tga

TGA(
  (gat): GenericGAT(
    (encoders): ModuleDict(
      (label): Linear(in_features=300, out_features=300, bias=True)
      (term): Linear(in_features=300, out_features=300, bias=True)
    )
    (layers): ModuleList(
      (0): GATConv(
        (fc): Linear(in_features=300, out_features=1200, bias=False)
        (feat_drop): Dropout(p=0.3, inplace=False)
        (attn_drop): Dropout(p=0.3, inplace=False)
        (leaky_relu): LeakyReLU(negative_slope=0.2)
        (res_fc): Identity()
      )
      (1): GATConv(
        (fc): Linear(in_features=300, out_features=1200, bias=False)
        (feat_drop): Dropout(p=0.3, inplace=False)
        (attn_drop): Dropout(p=0.3, inplace=False)
        (leaky_relu): LeakyReLU(negative_slope=0.2)
        (res_fc): Identity()
      )
    )
  )
  (norm_label): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (norm_terms): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (gate): Linea

In [21]:
loss_func_global = CosineDistanceLoss(reduction='mean', alpha=-1).to(device)
loss_func_local  = CosineDistanceLoss(reduction='mean').to(device)

optimizer = optim.AdamW( tga.parameters(), lr=5e-3, weight_decay=5e-3)


#RMSprop

In [22]:

best = None
nepochs = 25
lr = 2.*5e-2
best_loss = None
best_loss_local = None
best_loss_global = None
for e in tqdm(range(nepochs), total=nepochs):
    epoch_loss = 0
    data_loader = DataLoader(list(zip(fold.X_train, fold.y_train)), batch_size=batch_size,
                             shuffle=True, collate_fn=collate, num_workers=6)
    total_loss = 0.
    c_loss = 0.
    g_loss = 0.
    l_loss_pos = 0.
    l_loss_neg = 0.
    with tqdm(total=len(fold.y_train)) as pbar:
        total = 1
        correct = 1
        tga.train()
        for i, (G, gs, y) in enumerate(data_loader):
            G = G.to( device )
            gs = gs.to( device )
            y = y.to( device )
            
            h_docs, docs_preds, labels_hiddens = tga( G, gs, y )
            
            loss, expl = tga.loss(h_docs, docs_preds, labels_hiddens, y, explain=True)
            y_pred     = tga.loss.predict(h_docs, labels_hiddens).argmax(dim=1)
            
            loss.backward()

            optimizer.step()
            
            total_loss += loss.item()
            correct    += (y_pred == y).sum()
            total      += len(y)
            
            cross_ent   = expl['cross_entropy'][0]
            c_loss     += cross_ent
            
            global_     = expl['global'][0]
            g_loss     += global_
            
            local_pos_  = expl['local_pos'][0]
            l_loss_pos += local_pos_
            
            local_neg_  = expl['local_neg'][0]
            l_loss_neg += local_neg_
            
            
            
            to_print  = f'Acc: {(1.*correct/total).item():.3}, '
            to_print += f"cross_ent({expl['cross_entropy'][1]:.3}): {(c_loss/(i+1)):.3}, "
            to_print += f"global (sim, {expl['global'][1]:.3}): {(g_loss/(i+1)):.3}, "
            to_print += f"local (pos, dist, {expl['local_pos'][1]:.3}): {(l_loss_pos/(i+1)):.3}, "
            to_print += f"local (neg, sim, {expl['local_neg'][1]:.3}): {(l_loss_neg/(i+1)):.3}            "
            
            print( to_print , end='\r')
            pbar.update( len(y) )
            pbar.set_description_str(f'iter {e} Loss: {total_loss/(i+1):.6}')
            
            del loss, labels_hiddens, G, gs, y, h_docs, docs_preds
            #break
        #break

AttributeError: module 'torch.nn' has no attribute 'softmax'

In [33]:
docs_preds

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0.,

In [ ]:
margin=.0
loss_tga = TGALoss(margin=margin)
loss, explanation = loss_tga(h_local, labels_hiddens, y, explain=True)
print(loss)
print(explanation)
y_probs = loss_tga.predict(h_local, labels_hiddens)
y_preds = y_probs.argmax(dim=1)
print( (y == y_preds).sum().item() )

In [ ]:
big_graph_dgl = dgl.DGLGraph()
big_graph_dgl.from_networkx(graph_builder.g, node_attrs=['emb', 'label', 'idx'] )
label_idx = big_graph_dgl.ndata['label'].nonzero().flatten()


In [ ]:
best_h_global[label_idx] # Embedding dos labels (Distâncias maximizadas)

In [ ]:
                        # embedding dos termos novos
best_h_global[len(label_idx):] # o mapeamento é (o inverso de) node_mapper[big_graph.ndata['idx']]

In [ ]:
h_best[100:105]

In [ ]:
big_graph_dgl.ndata['emb'][100:105]

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import scatter

In [ ]:
# Variância média DAS hidden dimensions dos labels &
#     -> O quanto as hidden estão variando entre as labels
#        Quanto maior esse valor, mais "diferente" são as representações dos LABELS
# Variância média ENTRE A MÉDIA DAS hidden dimensions dos labels
#     -> O quanto as hiddens estão variando entre si
#        Quanto maior esse valor, mais "diferente" são as representações das DIMENSÕES
#        Ou seja, VAR(AVG(h1), AVG(h2), ..., AVG(hn)) é alto
best_h_global[label_idx].std(axis=0).mean(), best_h_global[label_idx].mean(axis=0).std()

In [ ]:
nbins = 30

In [ ]:
plt.hist( best_h_global[label_idx].std(axis=0).detach().cpu().numpy(), bins=nbins )
plt.title( 'Labels\n[VAR(h1), VAR(h2), ..., VAR(hn)]' )

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(verbose=10, metric='cosine')
X = tsne.fit_transform(best_h_global.detach().cpu().numpy())

In [ ]:
dataset.dname

In [ ]:
x,y = X.T
plt.figure(figsize=(10,8))
plt.scatter(x[len(label_idx):],y[len(label_idx):], linewidths= 0.0, s=25, alpha=0.025)
plt.scatter(x[label_idx.cpu().numpy()],y[label_idx.cpu().numpy()], marker='x')
plt.xlim( (x.min()-5, x.max()+5) )
plt.ylim( (y.min()-5, y.max()+5) )
plt.title(f'{dataset.dname} - heads={n_heads}')
plt.savefig(f'{dataset.dname}-heads={n_heads}_diff.png')

In [ ]:
from sklearn.manifold import TSNE
tsne = TSNE(verbose=10)
X2 = tsne.fit_transform(big_graph_dgl.ndata['emb'][len(label_idx):].detach().cpu().numpy())

In [ ]:
x,y = X2.T
plt.scatter(x, y, linewidths= 0.0, alpha=0.01)
plt.xlim( (x.min()-5, x.max()+5) )
plt.ylim( (y.min()-5, y.max()+5) )

In [ ]:
x[label_idx.cpu().numpy()],y[label_idx.cpu().numpy()]

In [ ]:
plt.hist( h_best[len(label_idx):].std(axis=0).detach().cpu().numpy(), bins=nbins )
plt.title( 'Terms\n[VAR(h1), VAR(h2), ..., VAR(hn)]' )

In [ ]:
plt.hist( h_best[label_idx].mean(axis=0).detach().cpu().numpy(), bins=nbins )
plt.title( 'Labels\n[AVG(h1), AVG(h2), ..., AVG(hn)]' )

In [ ]:
plt.hist( h_best[len(label_idx):].mean(axis=0).detach().cpu().numpy(), bins=nbins )
plt.title( 'Terms\n[AVG(h1), AVG(h2), ..., AVG(hn)]' )

In [ ]:
label_idx = big_graph_dgl.ndata['label'].nonzero().flatten()

In [ ]:
h = gat(big_graph_dgl)
h.shape

In [ ]:
labels_hiddens = h[label_idx]
labels_hiddens

In [ ]:
A = []
B = []
y = []
for i in range(labels_hiddens.size()[0]):
    for j in range(labels_hiddens.size()[0]):
        if i != j:
            A.append( labels_hiddens[i] )
            B.append( -1.*labels_hiddens[j] )
            y.append( 1 )
            
B=torch.cat(B).reshape( len(y), 300 )
A=torch.cat(A).reshape( len(y), 300 )
y=torch.Tensor(y)

In [ ]:
loss_func(B, A, target=y), loss_func(A, B, target=y)

In [ ]:
gat.train()
torch.cuda.synchronize()

In [ ]:
class TGA(nn.Module):
    def __init__(self, in_dim, hidden_dim, n_class,
                  n_heads=8, drop=.5, attn_drop=.5,
                  device='cuda:0'):
        super(TGA, self).__init__()
        self.n_class = n_class
        self.device = torch.device(device)
        self.gat_global = GenericGAT(in_dim, hidden_dim, 
                                     encoders={'label'}, 
                                     n_heads=n_heads, drop=drop,
                                     attn_drop=attn_drop, device=self.device)
        
        self.gat_local  = GenericGAT(hidden_dim, hidden_dim, 
                                     encoders={'term'}, 
                                     n_heads=n_heads, drop=drop,
                                     first_hidden='emb',
                                     attn_drop=attn_drop, device=self.device)

        self.lin = nn.Linear( hidden_dim, 1).to(self.device)
        # Depois tentar alguma ativação (ReLU, por exemplo, pode "desativar" alguns termos no softmax)
        self.pooling = GlobalAttentionPooling( self.lin ).to(self.device)

        # Fully Connected
        self.fc1 = nn.Linear( hidden_dim, hidden_dim//2).to(self.device)
        self.fc2 = nn.Linear( hidden_dim//2, hidden_dim//4).to(self.device)
        self.fc3 = nn.Linear( hidden_dim//4, self.n_class).to(self.device)
    def forward(self, G, gs):
        #h_global           = self.gat_global( G, label=G.ndata['label'].nonzero().flatten() )
        #gs.ndata['weight'] = h_global[ gs.ndata['idx'] ] # Tentar concatenando
        h_local            = self.gat_local(gs, term=None)
        #h_local            = torch.cat((h_local, h_global[ gs.ndata['idx'] ]), 1)
        h_local            = self.pooling( gs, h_local )
        h_local            = self.fc1( h_local )
        h_local            = self.fc2( h_local )
        h_local            = self.fc3( h_local )
        return h_local
# torch.Size([3652, 300]) torch.Size([3652, 300]) torch.Size([128, 300])
        

In [ ]:
in_dim=300
hidden_dim=2
n_heads=8
drop=0.3
attn_drop=0.5
batch_size=128
device='cuda:0'

In [ ]:
model = TGA( in_dim, hidden_dim, graph_builder.n_class,
            n_heads=n_heads, drop=drop, attn_drop=attn_drop )
model

In [ ]:
loss_func = nn.CrossEntropyLoss()

optimizer = optim.Adam( model.parameters(), lr=1e-3, weight_decay=1e-3)
#optimizer = optim.AdamW( model.parameters(), lr=1e-2, weight_decay=1e-3)

#optimizer = optim.RMSprop( model.parameters(), lr=1e-2, weight_decay=1e-4)
#optimizer = optim.RMSprop( model.parameters(), lr=0.0001 )

model.train()
torch.cuda.synchronize()

In [ ]:
g = nx.Graph()
g.add_nodes_from( [ (0, {'idx': 0}), (1, {'idx': 1}), (2, {'idx': 2}) ] )

In [ ]:
list(nx.get_node_attributes(g,'idx').values())

In [ ]:
outputs

In [ ]:
labels_hiddens = torch.eye( 11 )
labels_hiddens